In [1]:
from drn_interactions.interactions.preprocessors import InteractionsPreprocessor
from drn_interactions.interactions.loaders import SpontaneousActivityLoader
from drn_interactions.interactions.pairwise import PairwiseCorr
from drn_interactions.io import load_neurons
import seaborn as sns
import pandas as pd
import numpy as np
from drn_interactions.interactions.graph_clustering import SpectralCluster, df_to_graph, ClusterRunner, ClusterEvaluation
import warnings
from drn_interactions.config import Config



In [2]:

def reorder_df_affinity(df, ensemble_labels):
    idx = np.argsort(ensemble_labels)
    return df.iloc[idx, idx]

In [3]:
sessions = load_neurons()["session_name"].unique()

preprocessor = InteractionsPreprocessor(z=True, minmax=False)
evaluator = ClusterEvaluation()

loader_fac = lambda bin_width, session, shuffle: SpontaneousActivityLoader(
    bin_width=bin_width, session_name=session, shuffle=shuffle
)
corr_fac = lambda: PairwiseCorr(
    remove_self_interactions=True,
    rectify=True,
    shuffle=False,
    remove_duplicate_combs=True,
)
cluster_fac = lambda: SpectralCluster(
    n_clusters=np.arange(2, 8), spectral_kws={"affinity": "precomputed"}, n_init=20
)

runner_fac = lambda bin_width, session, shuffle_load: ClusterRunner(
    loader=loader_fac(bin_width, session, shuffle_load),
    preprocessor=preprocessor,
    affinity_calculator=corr_fac(),
    clusterer=cluster_fac(),
    evalulator=evaluator,
)


In [4]:
ensemble_frames = []
ensemble_stats_frames = []
current_max = 1
bin_widths = (0.05, 0.1, 0.5, 1, 2)
nboot = 50

for shuffle in (False, True):
    for bin_width in bin_widths:
        for session in sessions[:3]:

            # calculate ensembles
            runner = runner_fac(bin_width, session, shuffle)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                _, ensembles, _, ensemble_stats = runner.run(20)

            # update ensembles to global max
            ensembles["ensemble"] = np.where(
                ensembles["ensemble"] != -1, ensembles["ensemble"] + current_max, -1
            )
            ensemble_stats["ensemble"] = np.where(
                ensemble_stats["ensemble"] != -1,
                ensemble_stats["ensemble"] + current_max,
                -1,
            )
            current_max = ensembles["ensemble"].abs().max() + 1

            # add metadata and append to list of results
            ensembles = ensembles.assign(
                session_name=session, shuffle=shuffle, bin_width=bin_width
            )
            ensemble_stats = ensemble_stats.assign(
                session_name=session, shuffle=shuffle, bin_width=bin_width
            )
            ensemble_stats_frames.append(ensemble_stats)
            ensemble_frames.append(ensembles)


df_ensembles = pd.concat(ensemble_frames).reset_index(drop=True)
df_ensembles_stats = pd.concat(ensemble_stats_frames).reset_index(drop=True)


In [15]:
dd = Config.derived_data_dir / "ensembles"
dd.mkdir(exist_ok=True)
df_ensembles.to_csv(dd / "spont - ensembles.csv", index=False)
df_ensembles.to_csv(dd / "spont - stats.csv", index=False)